### Importamos 

In [13]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os


### Variables de entorno


In [14]:
load_dotenv()
api_key = os.getenv("API_KEY")  

### Subimos el documento

In [15]:

def upload_pdf(url: str):        
    try:
        loader = PyPDFLoader(url)
        loader = loader.lazy_load()

        text = ""

        for page in loader: 
            text += page.page_content + "\n"

        return text
    except Exception as e:
        print(e)
        return []
    


## Text Splitter para separar todo el contenido de mi documento
### aumento del chunk size para la obtención de oraciones más largas
### Decremento del chunk_overlap para intentar hacerle perder el contexto al modelo, diviendo las frases importantes para que pierda la relación entre las palabras

- CharacterTextSplitter es más directo, divide el texto en trozos pequeños fijos.
- Aquí utilicé RecursiveCharacterTextSplitter que según la documentación es más inteligente y jerarquico, ideal para PDFs, libros o documentación

In [16]:

def text_splitter(text): 

    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 30
    )
    texts = text_splitter.create_documents([text])
    print(texts)
    return texts



# Defino el modelo que utilizaré

1. Nuevo modelo de embedding de ollama (mxbai-embed-large): Es un modelo de incrustación (embedding) de lenguaje de última generación, de código abierto, desarrollado por Mixedbread.ai. 
- Su función principal es transformar texto (palabras, frases o documentos) en representaciones numéricas de alta dimensión, conocidas como vectores. Estos vectores capturan el significado semántico y sintáctico del texto, lo que permite a los sistemas de IA comprender la relación y similitud entre diferentes fragmentos de texto. 

* Este modelo solicitó otra vector Store ya que el anterior tenia un soporte para 768 dimensiones y este tiene hasta 1024 dimensiones al tratarse de un modelo de embedding más grande. Entonces, ¿a qué me refiero con dimensiones? la cantidad de números con los que cuenta un vector creado por el embedding

In [17]:
embedding = OllamaEmbeddings(
    model = "mxbai-embed-large:latest"
)

### Creo mi base de datos vectorial donde se guardará mi embedding

In [18]:
def get_vector_store(name_collection: str): 
    
    vector_store = Chroma(
    collection_name= name_collection,
    embedding_function=embedding,
    persist_directory="./prueba_chroma"
)    
    return vector_store

### Creo el retrieval que devolverá la información en una busqueda de similitudes

In [19]:
def retrieval(input_user: str): 
    vector_store = get_vector_store("langchainPrueba")
    docs = vector_store.similarity_search(input_user)
    return docs

### Creamos el propt system para el modelo

In [20]:
prompt = PromptTemplate.from_template("""
    Eres un asistente encargado de responder preguntas sobre Arquitectura de software y solo debes contestar si el contexto no está vacio.
    En caso de que no cuentes con la información solicitada responde "La pregunta excede mi conocimiento" y si te preguntan algo fuera del contexto principal responde "No estoy programado para eso".
    Utiliza siempre el contexto proporcionado para responder y también utiliza un lenguaje familiar y amigable, con carisma.
    contexto = {contexto}
    pregunta del usuario: {input_user}
""")

### Creamos la función de respuesta que nos comunicará con nuestro agende de IA

In [21]:
def response(input_user: str, contexto: str):
    llm = ChatGoogleGenerativeAI(
    api_key=api_key,
    model="gemini-2.0-flash-lite",
    temperature= 0.5
)

    for chunk in llm.stream(prompt.format(contexto=contexto, input_user=input_user)):
        yield chunk.content

### Utilizamos las funciones para cargar el doc, aplicarle el text_splitter y guardar esos datos como embedding en la base de datos vectorial

In [22]:
loader = upload_pdf("mi_tp.pdf")
texts = text_splitter(loader)
vector_store = get_vector_store("langchainPrueba")

vector_store.add_documents(texts)

[Document(metadata={}, page_content='1. Investiga y explica con tus palabras cada uno de los siguientes conceptos(podés usar libros,'), Document(metadata={}, page_content='conceptos(podés usar libros, videos, artículos, papers o apuntes de clase):○ ¿Qué es una red'), Document(metadata={}, page_content='de clase):○ ¿Qué es una red neuronal artificial?Una red neuronal artificial es una red neuronal que'), Document(metadata={}, page_content='es una red neuronal que es artificial porque no es real, es creada por nosotros, es decir es creada'), Document(metadata={}, page_content='nosotros, es decir es creada por el humano pero no de nacimiento sino que fue creado por nosotros'), Document(metadata={}, page_content='que fue creado por nosotros (Son nodos que se encargan de realizar una predicción a partir de datos'), Document(metadata={}, page_content='predicción a partir de datos que se le brindan a través de tensores que pueden estar más o menos'), Document(metadata={}, page_content='que pu

['dbe97c3c-f730-4d14-8eed-dbdc3de0dfbc',
 'd945e283-5617-4fc4-becb-5f3b6724c7f3',
 '45412571-2b38-40d6-a9e3-2518bc950c0a',
 'f6d8e8b7-c51a-4932-9e39-40494b6c22be',
 '937b4ae1-668b-43f2-9626-69c015a9b3a1',
 '912c2928-83e1-4a1b-849a-3dfb27989d5a',
 '96e51cef-e4d6-4574-b461-8367a197c316',
 '8004f71c-acc9-4c04-99ac-af84995a9c4d',
 '46bb065a-2d49-499f-b0c6-aa118dd27f89',
 '7fb79e16-b569-40c5-a793-b282e079337c',
 'c5b5014d-6113-4696-88fe-e258dc276c72',
 'a0d408ab-5d4a-485a-b197-ddeb89ea5fed',
 '4305b668-ef22-4882-be84-c8984d488098',
 'c985e209-d56d-4324-9431-2457509dcda5',
 '79c5c1a6-dc90-4098-b32a-c0cbea85771d',
 '822e7d7d-0ee7-4a33-b8f9-39f81f6d0c17',
 '138f97c2-cf4a-4602-bd5a-884a66904335',
 '4eb29458-83a5-4de2-a919-262bbad4ed89',
 '55a9dda8-e602-48e7-841a-13ba96e813f7',
 '9c9c700b-657b-447b-b2e9-642f0c9fb559',
 'e02f2151-cfd9-4c65-81aa-e1c20eef9550',
 '830a36ea-9a28-4bb7-b1ff-386822affb6b',
 '22145d6c-e194-4e3c-a9da-d4d75f9bcf91',
 '06fbd9ac-9785-4e5e-847a-7488cfa3c7aa',
 'e0849177-4779-

### Ponemos a prueba nuestro RAG
ya que se disminuyo el chunck_size y el overlap nos damos cuenta luego de la primera respuesta ya pierde el contexto.

In [23]:

for l in range(3):
    input_user = input("Human: ")
    print(input_user)

    docs = retrieval(input_user=input_user)
    print(docs)

    for chunk in response(input_user=input_user, contexto=docs):
        print(chunk, end="", flush=True)
    

que es una neurona
[Document(id='b6d12b4c-d6a0-49b2-8ddd-4c1f8999ffe7', metadata={}, page_content='es el primer paso fundamental en el ciclo de entrenamiento de una red neuronal, después de que la'), Document(id='55a9dda8-e602-48e7-841a-13ba96e813f7', metadata={}, page_content='es el primer paso fundamental en el ciclo de entrenamiento de una red neuronal, después de que la'), Document(id='16b08668-ea45-4121-9ac6-ca5ff2fb19c7', metadata={}, page_content='es una función matemática que va a decidir si una neurona está activa o no, es decir, cuanta'), Document(id='06fbd9ac-9785-4e5e-847a-7488cfa3c7aa', metadata={}, page_content='es una función matemática que va a decidir si una neurona está activa o no, es decir, cuanta')]
¡Hola! Según el contexto que tengo, no puedo responder qué es una neurona. La información que tengo se enfoca en el primer paso del entrenamiento de una red neuronal y en una función matemática que decide la activación de una neurona. ¡Lo siento!
que es una funcion de a

### Al cambiar el chunck_size y el overlap dio una respuesta más concisa, al disminuirlo daba respuestas malas, no tan significativas y comprensibles

In [24]:

for l in range(3):
    input_user = input("Human: ")
    print(input_user)

    docs = retrieval(input_user=input_user)
    print(docs)

    for chunk in response(input_user=input_user, contexto=docs):
        print(chunk, end=" ", flush=True)
    

que es una neurona
[Document(id='b6d12b4c-d6a0-49b2-8ddd-4c1f8999ffe7', metadata={}, page_content='es el primer paso fundamental en el ciclo de entrenamiento de una red neuronal, después de que la'), Document(id='55a9dda8-e602-48e7-841a-13ba96e813f7', metadata={}, page_content='es el primer paso fundamental en el ciclo de entrenamiento de una red neuronal, después de que la'), Document(id='16b08668-ea45-4121-9ac6-ca5ff2fb19c7', metadata={}, page_content='es una función matemática que va a decidir si una neurona está activa o no, es decir, cuanta'), Document(id='06fbd9ac-9785-4e5e-847a-7488cfa3c7aa', metadata={}, page_content='es una función matemática que va a decidir si una neurona está activa o no, es decir, cuanta')]
¡ Hola! 😊 Según el contexto que tengo, no puedo responder qué es una neurona . La información que tengo se enfoca en el primer paso del entrenamiento de una red  neuronal y en una función matemática que decide la activación de una neurona.
  ¿Qué función cumplen las cap